### Question Answer Application
The goal of Question Answering is to find the answer to a question given a question and an accompanying context. The predicted answer will be either a span of text from the context or an empty string (indicating the question cannot be answered from the context).

In [1]:
!pip install simpletransformers

  Obtaining dependency information for simpletransformers from https://files.pythonhosted.org/packages/16/c8/20d7eede93e320c0746c01d2205bdbeb388c236247244b9428e743a96de9/simpletransformers-0.64.3-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/42.3 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.3 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.3 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.3 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.3 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.3 kB ? eta -:--:--
     ------------------ -------------------- 20.5/42.3 kB 43.6 kB/s eta 0:00:01
     ---------------------------- ---------- 30.7/42.3 kB 72.6 kB/s eta 0:00:01
     ---------------------------- ---------- 30.7/42.3 kB 72.6 kB/s eta 0:00:01
     ---------------------------- ---------- 30.7/42.3 kB 72.6 kB/s eta 0:00:01
     ----------------

In [6]:
import json
with open("total_dataset", "r") as read_file:
    train = json.load(read_file)

FileNotFoundError: [Errno 2] No such file or directory: 'total_dataset'

In [ ]:
total_dataset

In [ ]:
with open(r"total_dataset", "r") as read_file:
    test = json.load(read_file)

In [ ]:
total_dataset

In [ ]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [ ]:
model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [ ]:
# Configure the model 
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size=3
model_args.num_train_epochs=5


In [73]:
### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 5,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [ ]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args
)

In [74]:
### Remove output folder
!rm -rf outputs

In [3]:
# Train the model
model.train_model(train, eval_data=total_dataset)

NameError: name 'model' is not defined

In [77]:
# Evaluate the model
result, texts = model.eval_model(test)

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [78]:
# Make predictions with the model
to_predict = [
    {
        "context": "Vin is a Mistborn of great power and skill.",
        "qas": [
            {
                "question": "What is Vin's speciality?",
                "id": "0",
            }
        ],
    }
]

In [79]:
answers, probabilities = model.predict(to_predict)

print(answers)


convert squad examples to features: 100%|██████████| 1/1 [00:00<00:00, 4744.69it/s]

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 658.45it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '0', 'answer': ['is a Mistborn', 'is a Mi', 'born']}]
